# 1millionwomentotech SummerOfCode

## Intro to AI: Week 4 Day 3

In [35]:
print(baby_train[50000]['reviewText'])

I did hours of research on strollers... contemplating between the First Years Indigo and theBumbleride Flyer Reversible Handle Stroller with 7" Wheels, Seagrass. I finally settled on the Indigo stroller because the price was right and the features I wanted were all there. I purchased the stroller and matchingThe First Years Via Infant Car Seat, Abstract O'scar seat. My son was born in the winter and I didn't use the Indigo stroller regularly until about two months ago. For the first week, the stroller was wonderful. What I had initially liked most about the stroller was that I could easily attach the matching First Years Via car seat. In addition, the ride was smooth, quiet, and comfy. The stroller was a breeze to fold. Then, about two weeks into regular use, I started to notice the quality of the stroller diminish quickly. Here is what started to happen:(1) SQUEAKY WHEELS. The wheels started squeaking terribly. I'd oil them and within a few days the stroller was squeaking again! I'd b

In [36]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
text = baby_train[50000]['reviewText']
for s in sent_tokenize(text):
    print(s)
    print(sia.polarity_scores(s))

I did hours of research on strollers... contemplating between the First Years Indigo and theBumbleride Flyer Reversible Handle Stroller with 7" Wheels, Seagrass.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I finally settled on the Indigo stroller because the price was right and the features I wanted were all there.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I purchased the stroller and matchingThe First Years Via Infant Car Seat, Abstract O'scar seat.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
My son was born in the winter and I didn't use the Indigo stroller regularly until about two months ago.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For the first week, the stroller was wonderful.
{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}
What I had initially liked most about the stroller was that I could easily attach the matching First Years Via car seat.
{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.6369}
In addition, the ri

In [37]:
def sia_features(dataset):
    """For each review text in the dataset, extract:
       (1) the mean positive sentiment over all sentences
       (2) the mean neutral sentiment over all sentences
       (3) the mean negative sentiment over all sentences
       (4) the maximum positive sentiment over all sentences
       (5) the maximum neutral sentiment over all sentences
       (6) the maximum negative sentiment over all sentences"""
    feat_matrix = numpy.empty((len(dataset), 6))
    for i in range(len(dataset)):
        sentences = sent_tokenize(dataset[i]['reviewText'])
        nsent = len(sentences)
        if nsent:
            sentence_polarities = numpy.empty((nsent, 3))
            for j in range(nsent):
                polarity = sia.polarity_scores(sentences[j])
                sentence_polarities[j, 0] = polarity['pos']
                sentence_polarities[j, 1] = polarity['neu']
                sentence_polarities[j, 2] = polarity['neg']
            feat_matrix[i, 0:3] = numpy.mean(sentence_polarities, axis=0) # mean over the columns
            feat_matrix[i, 3:6] = numpy.max(sentence_polarities, axis=0) # maximum over the columns
        else:
            feat_matrix[i, 0:6] = 0.0
    return feat_matrix

sia_tr = sia_features(baby_train)

In [38]:
testmat = numpy.arange(12.).reshape((3, 4))
print(testmat)
print(numpy.max(testmat, axis=0))
print(numpy.mean(testmat, axis=1))

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[ 8.  9. 10. 11.]
[1.5 5.5 9.5]


In [39]:
def len_features(dataset):
    """Add two features:
       (1) length of review (in thousands of characters) - truncate at 2,500
       (2) percentage of exclamation marks (in %)"""
    feat_matrix = numpy.empty((len(dataset), 2))
    for i in range(len(dataset)):
        text = dataset[i]['reviewText']
        feat_matrix[i, 0] = len(text) / 1000.
        if text:
            feat_matrix[i, 1] = 100. * text.count('!') / len(text)
        else:
            feat_matrix[i, 1] = 0.0
    feat_matrix[feat_matrix>2.5] = 2.5
    return feat_matrix

len_tr = len_features(baby_train)

In [40]:
print(X_train_neg.shape, sia_tr.shape, len_tr.shape)

(96512, 2) (96512, 6) (96512, 2)


In [41]:
X_train_augmented = numpy.concatenate((X_train_neg, sia_tr, len_tr), axis=1) # stack horizontally
lreg_augmented = LinearRegression().fit(X_train_augmented, Y_train)
pred_train_augmented = lreg_augmented.predict(X_train_augmented)
mae_train_augmented = mean_absolute_error(pred_train_augmented, Y_train)
print("Now the mean absolute error on the training data is %f stars" % mae_train_augmented)

Now the mean absolute error on the training data is 0.758256 stars


In [42]:
rf_augmented = RandomForestRegressor().fit(X_train_augmented, Y_train)
rfpred_train_augmented = rf_augmented.predict(X_train_augmented)
mae_train_rf_augmented = mean_absolute_error(rfpred_train_augmented, Y_train)
print("For the RF, it is %f stars" % mae_train_rf_augmented)

For the RF, it is 0.283528 stars


In [43]:
X_valid_neg = dataset_to_matrix_with_neg(baby_valid)
sia_valid = sia_features(baby_valid)
len_valid = len_features(baby_valid)
X_valid_augmented = numpy.concatenate((X_valid_neg, sia_valid, len_valid), axis=1)
pred_valid_augmented = lreg_augmented.predict(X_valid_augmented)
pred_valid_rf_augmented = rf_augmented.predict(X_valid_augmented)
mae_valid_augmented = mean_absolute_error(pred_valid_augmented, Y_valid)
print("On the validation set, we get %f error for the linear regression" % mae_valid_augmented)
mae_valid_rf_augmented = mean_absolute_error(pred_valid_rf_augmented, Y_valid)
print("And %f for the random forest regression" % mae_valid_rf_augmented)

On the validation set, we get 0.755795 error for the linear regression
And 0.731631 for the random forest regression


In [44]:
print(baby_train[50000]['reviewText'])

I did hours of research on strollers... contemplating between the First Years Indigo and theBumbleride Flyer Reversible Handle Stroller with 7" Wheels, Seagrass. I finally settled on the Indigo stroller because the price was right and the features I wanted were all there. I purchased the stroller and matchingThe First Years Via Infant Car Seat, Abstract O'scar seat. My son was born in the winter and I didn't use the Indigo stroller regularly until about two months ago. For the first week, the stroller was wonderful. What I had initially liked most about the stroller was that I could easily attach the matching First Years Via car seat. In addition, the ride was smooth, quiet, and comfy. The stroller was a breeze to fold. Then, about two weeks into regular use, I started to notice the quality of the stroller diminish quickly. Here is what started to happen:(1) SQUEAKY WHEELS. The wheels started squeaking terribly. I'd oil them and within a few days the stroller was squeaking again! I'd b

In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
text = baby_train[50000]['reviewText']
for s in sent_tokenize(text):
    print(s)
    print(sia.polarity_scores(s))

I did hours of research on strollers... contemplating between the First Years Indigo and theBumbleride Flyer Reversible Handle Stroller with 7" Wheels, Seagrass.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I finally settled on the Indigo stroller because the price was right and the features I wanted were all there.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I purchased the stroller and matchingThe First Years Via Infant Car Seat, Abstract O'scar seat.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
My son was born in the winter and I didn't use the Indigo stroller regularly until about two months ago.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For the first week, the stroller was wonderful.
{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}
What I had initially liked most about the stroller was that I could easily attach the matching First Years Via car seat.
{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.6369}
In addition, the ri

In [53]:
def sia_features(dataset):
    """For each review text in the dataset, extract:
    (1) mean positive sentiment over all sentences
    (2) mean neutral sentiment over all sentences
    (3) mean negative sentiment over all sentences
    (4) maximum positive sentiment over all sentences
    (5) maximum neutral sentiment over all sentences
    (6) maximum negative sentiment over all sentences
    """
    feat_matrix = numpy.empty((len(dataset), 6))
    for i in range(len(dataset)):
        sentences = sent_tokenize(dataset[i]['reviewText'])
        nsent = len(sentences)
        if nsent:
            sentence_polarities = numpy.empty((nsent, 3))
            for j in range(nsent):
                polarity = sia.polarity_scores(sentences[j])
                sentence_polarities[j, 0] = polarity['pos']
                sentence_polarities[j, 1] = polarity['neu']
                sentence_polarities[j, 2] = polarity['neg']
            feat_matrix[i, 0:3] = numpy.mean(sentence_polarities, axis = 0) # mean over the columns
            feat_matrix[i, 3:6] = numpy.max(sentence_polarities, axis = 0) # maximum over the columns
        else:
            feat_matrix[i, 0:6] = 0.0
    return feat_matrix    
            
sia_tr = sia_features(baby_train)
print(sia_tr[:10])

[[0.23533333 0.76466667 0.         0.552      1.         0.        ]
 [0.0198     0.9678     0.0124     0.099      1.         0.062     ]
 [0.12725    0.8365     0.03625    0.194      0.865      0.145     ]
 [0.1595     0.7775     0.063      0.319      0.874      0.126     ]
 [0.073      0.927      0.         0.219      1.         0.        ]
 [0.3115     0.6885     0.         0.571      1.         0.        ]
 [0.172      0.8174     0.0106     0.398      1.         0.053     ]
 [0.1035     0.87575    0.02075    0.279      1.         0.096     ]
 [0.29133333 0.70866667 0.         0.406      0.787      0.        ]
 [0.106      0.894      0.         0.318      1.         0.        ]]


In [49]:
testmat = numpy.arange(12.).reshape((3,4))
print(testmat)
print(numpy.max(testmat, axis = 0))
print(numpy.mean(testmat, axis = 1))

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[ 8.  9. 10. 11.]
[1.5 5.5 9.5]


In [ ]:
# Homework - required for Certification

def len_features(dataset):
    """Add two features:
    (1) length of review (in thousands of character) - truncate at 2,500
    (2) percentage of exclamation marks (in %)
    """

len_tr = len_features(baby_train)    
 

In [50]:
print(X_train_neg.shape, sia_tr.shape)

(96512, 2) (96512, 6)


In [55]:
# stack horizontally
X_train_augmented = numpy.concatenate( (X_train_neg, sia_tr), axis = 1)
lreg_augmented = LinearRegression().fit(X_train_augmented, Y_train)
pred_train_augmented = lreg_augmented.predict(X_train_augmented)
mae_train_augmented = mean_absolute_error(pred_train_augmented, Y_train)
print("Now the mean absolute error on the training data is %f starts" % mae_train_augmented)

Now the mean absolute error on the training data is 0.759126 starts


In [56]:
# random forest
rf_augmented = RandomForestRegressor().fit(X_train_augmented, Y_train)
rfpred_train_augmented = rf_augmented.predict(X_train_augmented)
mae_train_rf_augmented = mean_absolute_error(rfpred_train_augmented, Y_train)
print("For the RF, MAE is %f stars" % mae_train_rf_augmented)

For the RF, MAE is 0.292433 stars


In [ ]:
X_valid_neg = dataset_to_matrix_with_neg(baby_valid)
sia_valid = sia_features(baby_valid)
# len_valid = 
X_valid_augmented = numpy.concatenate((X_valid_neg, sia_valid), axis = 1)
pred_valid_augmented =
pred_valid_rfaugmented =

mae_valid_augmented = 
mae_valid_rfaugmented =

# Homework for certification

Refactor the code above:
- "Be lazy. Not just lazy but proactively, agressively lazy." Remove duplication.
- create a single function that takes in data and spits out all success metrics across all of your algos.


# Where to go from here?


- unigrams (NLTK)
- word vector (gensim, [glove](https://nlp.stanford.edu/projects/glove/), word2vec)
- recurrent neural net
- convolutional neural net

https://www.oreilly.com/learning/perform-sentiment-analysis-with-lstms-using-tensorflow

http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/

https://machinelearningmastery.com/develop-n-gram-multichannel-convolutional-neural-network-sentiment-analysis/